# Part 4
Now we apply our solution(s) from part 3 to the files in the scans folder.
Again, we use our 'pypdf2' txt files.

In [17]:
# Imports
# io
import os
import shutil

# analyze
import re

In [7]:
files = []

for file in os.listdir(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'files', 'scans')):
    name, ext = os.path.splitext(file)
    if ext == '.pdf':
        path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'files', 'scans', file)
        files.append(os.path.join(os.path.dirname(path), os.path.splitext(file)[0] + "_" + "pypdf2" + ".txt"))

files_scans = [file for file in files if "scans" in file]
files_scans_copy = []

In [10]:
def copy_scan_files():
    """
    Copies the txt files used for the analysis. Keeps the file name, adds 'copy'.
    """
    for _ in files_scans:
        shutil.copy(_, _.replace('.txt','_copy.txt'))
        files_scans_copy.append(_.replace('.txt', '_copy.txt'))

In [11]:
copy_scan_files()

Let's copy over our functions we used to find phone numbers from part3.ipynb

We will perform some preformatting, otherwise this test will be pretty pointless.
Since we have different methods for each file in part 3 we will show the best one here.

In [14]:
def format_scan_files():
    for txt_file in files_scans_copy:
        text = ""
        with open(txt_file, 'r') as scan1_file:
            text += scan1_file.read()
        text = text.replace('\n', ' ')  # Get rid of newlines
        text = text.replace('-', ' ')  # Get rid of dashes
        text = text.replace('/', ' ')  # Get rid of slashes
        text = text.replace('(', '')  # Get rid of parentheses
        text = text.replace(')', '')
        text = text.replace(' ', '')  # Get rid of spaces
        with open(txt_file, 'w') as scan1_file:
            scan1_file.writelines(text)

In [15]:
format_scan_files()

In [18]:
numbers = []

In [19]:
# From part3: We used this approach with the 'wegweiser senioren' file as well.
# We did not use it for the bundeswehr file because we got way better results using a different method.
def find_numbers_bahnstadt(path_to_file) -> list:
    """
    Searches inside the bahnstadt copy txt file for valid phone numbers.
    :return: List of numbers found in txt file.
    """
    matches = []
    fax_numbers = []
    bahnstadt_copy = path_to_file
    text = ""
    with open(bahnstadt_copy, 'r') as bahnstadt_file:
        text += bahnstadt_file.read()
    for match in re.findall(r'(110|112|115|116117|19222|116006)', text):
        matches.append(match)
    for match in re.findall(r'0[1-9][0-9]{6,12}', text):
        matches.append(match)
    for match in re.findall(r'[a-zA-Z]*fax[a-zA-Z]*(\d+)', text, flags=re.I):  # find all fax numbers
        fax_numbers.append(match)
    matches = [match for match in matches if match not in fax_numbers]  # remove fax numbers from solution
    return list(set(matches))  # convert to set to remove duplicates

Let's see how many numbers we can find:

In [22]:
for scan_file in files_scans_copy:
    for num in find_numbers_bahnstadt(scan_file):
        numbers.append(num)

In [23]:
print(len(numbers))

54


In [24]:
print(numbers)

['0622160430', '062211383620', '0622197370', '06202859430', '115', '062215850930', '062211383613', '0622150259595', '06221140714', '01711734982', '018122008708', '01726239609', '01751699490', '062219831331', '01712819736', '015158778269', '01788957264', '015773565090', '01773434363', '110', '115', '01707781800', '01775987162', '017623232774', '112', '01791173263', '017632137783', '0622160430', '062211383620', '0622197370', '06202859430', '115', '062215850930', '062211383613', '0622150259595', '06221140714', '01711734982', '018122008708', '01726239609', '01751699490', '062219831331', '01712819736', '015158778269', '01788957264', '015773565090', '01773434363', '110', '115', '01707781800', '01775987162', '017623232774', '112', '01791173263', '017632137783']


## Analyze our findings
### What kind of numbers did we find?
We found 54 numbers which have two different formats.
First, we have the short 'emergency' type numbers like 115, 112, which we capture explicitly.
Then, we have numbers that begin with 0, then a non-zero number and then 6 to 12 more numbers.


---

Obviously, we miss out on a large amount of phone numbers since we don't capture phone numbers that are not prefixed by their 'Vorwahl'.